<a href="https://colab.research.google.com/github/yangju0411/opinion_poll/blob/main/nesdc_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
import time

# 리얼미터 크롤링

## 사전 정보

조사기관이 리얼미터이고 여론조사명이 전국 정기(정례)조사 혹은 전국 정례조사인 여론조사의 응답률을 크롤링할 것이다.

In [4]:
url = "https://www.nesdc.go.kr/portal/bbs/B0000005/list.do?menuNo=200467"

중앙여론조사심의위원회의 여론조사결과현황 메뉴 url이다.

홈페이지 구조 상 조사기관명과 여론조사명을 동시에 검색하지 못 한다. 따라서 리얼미터의 여론조사를 검색한 후 그 중에서 조사명이 "전국 정기(정례)조사"(18년 8월 이후) 혹은 "전국 정례조사"(18년 8월 이전)인 것을 골라내야 한다.

우선 리얼미터로 검색했을 때 결과 페이지가 몇 개가 나타나는지를 먼저 수집해야한다.

검색할 때 조정할 패러미터 중 필요한 것만 간추렸다.

**searchTime** <br>
1: 등록일 <br>
2: 최초공표일 <br>
3: 조사일시 <br>

<br>
sdate: 시작
edate: 끝 <br>
yyyy-mm-dd 형식

**searchCnd** <br>
1: 조사기관명<br>
3: 여론조사명칭<br>

**searchWrd** <br>
searchCnd에서 선택한대로 검색할 검색어

In [5]:
rm_params = {"searchTime" : 1, "sdate": "2017-05-17", "edate": "", "searchCnd": "1", "searchWrd": "리얼미터", "pageIndex" : ""}

https://namu.wiki/w/%EB%AC%B8%EC%9E%AC%EC%9D%B8/%EC%A7%80%EC%A7%80%EC%9C%A8 <br>
나무위키 문재인 지지율 항목에 나와있는 국정수행평가 데이터와 연동할 것이므로 2017년 5월 3주차 즉 5월 17일부터 검색을 실시한다.

In [6]:
rm_url = requests.get(url, rm_params)
rm_text = rm_url.text
rm_soup = bs(rm_text, 'html.parser')

beautifulsoup 객체로 저장한다.

In [7]:
lastpage_reg = 'pageIndex=([0-9]*)\'\" type=\"button\">맨끝</button></span>'

정규 표현식을 통해 마지막 페이지의 인덱스를 뽑을 것이다.

In [8]:
rm_lp = int(re.search(lastpage_reg, str(rm_soup)).group(1))
print(rm_lp)

72


총 72페이지가 존재한다.

## 수집

In [9]:
soup_list = []
soup_append = soup_list.append

파이썬에서 반복문 안에서 객체의 메소드를 호출하면 느리다길래 한 번 해보았다.

In [10]:
%%time
for index in range(1, rm_lp + 1):
  rm_params['pageIndex'] = index
  rm_url = requests.get(url, rm_params)
  rm_text = rm_url.text
  rm_soup = bs(rm_text, 'html.parser').select("body section div div div.board div.grid a")
  soup_append(rm_soup)

CPU times: user 2.68 s, sys: 112 ms, total: 2.79 s
Wall time: 1min 32s


파싱한 것들을 저장한다.
필요한 a 부분만 추출한다.

In [11]:
rm_df = pd.DataFrame({"date" : [], "sname" : [], "href" : [], "resRate" : []})

In [12]:
rm_df

,date,sname,href,resRate


In [13]:
resRate_reg = '<th>전체 응답률</th>\n<td>[ \t\n\r\f\v]*(\d*\.?\d+)%'

응답률을 뽑기 위한 정규표현식

In [14]:
%%time
for soup in soup_list:
  for in_soup in soup:
    if "정례조사" in in_soup.select("span")[3].text or "정기(정례)조사" in in_soup.select("span")[3].text:
      if "주간집계" in in_soup.select("span")[3].text:
        rm_df = rm_df.append({"date": in_soup.select("span")[4].text, "sname": in_soup.select("span")[3].text,
                      "href": in_soup["href"]}, ignore_index = True)

CPU times: user 809 ms, sys: 27.9 ms, total: 837 ms
Wall time: 800 ms


In [15]:
def crawlResrate(href):
  url = "https://www.nesdc.go.kr"
  text = requests.get(url + href).text
  soup = bs(text, "html.parser")
  rate = re.search(resRate_reg, str(soup)).group(1)
  return rate

각 문서 별 응답률을 크롤링해오는 함수

In [16]:
%%time
rm_df["resRate"] = rm_df.apply(lambda x:crawlResrate(x["href"]), axis = 1)

CPU times: user 14.2 s, sys: 241 ms, total: 14.5 s
Wall time: 4min 34s


행 별로 href를 뽑아와 응답률을 구하는 코드

In [17]:
rm_df["resRate"] = rm_df["resRate"].map(float)

In [18]:
rm_df

,date,sname,href,resRate
0,2020-11-27,전국 정기(정례)조사 정당지지도 11월 4주차 주간집계,/portal/bbs/B0000005/view.do?nttId=8113&menuNo...,4.4
1,2020-11-20,전국 정기(정례)조사 정당지지도 11월 3주차 주간집계,/portal/bbs/B0000005/view.do?nttId=8107&menuNo...,3.7
2,2020-11-13,전국 정기(정례)조사 정당지지도 11월 2주차 주간집계,/portal/bbs/B0000005/view.do?nttId=8092&menuNo...,4.8
3,2020-11-06,전국 정기(정례)조사 정당지지도 11월 1주차 주간집계,/portal/bbs/B0000005/view.do?nttId=8086&menuNo...,4.5
4,2020-10-30,전국 정기(정례)조사 정당지지도 10월 4주차 주간집계,/portal/bbs/B0000005/view.do?nttId=8073&menuNo...,4.3
...,...,...,...,...
182,2017-06-16,전국 정례조사 6월 2주 주간집계,/portal/bbs/B0000005/view.do?nttId=3826&menuNo...,5.4
183,2017-06-09,전국 정례조사 6월 1주 주간집계,/portal/bbs/B0000005/view.do?nttId=3819&menuNo...,5.3
184,2017-06-02,전국 정례조사 5월 5주 주간집계,/portal/bbs/B0000005/view.do?nttId=3813&menuNo...,6.0
185,2017-05-26,전국 정례조사 5월 4주 주간집계,/portal/bbs/B0000005/view.do?nttId=3805&menuNo...,6.8


In [19]:
rm_df["resRate"].isnull().sum()

0

다행히 결측값 없이 응답률이 잘 크롤링 되었다.

In [20]:
rm_df["resRate"].mean()

5.53636363636364

In [21]:
rm_df.shape

(187, 4)

In [23]:
rm_df.to_csv("/content/drive/MyDrive/여론조사/리얼미터_응답률.csv")